# CSIS4260_Project
## Movie Recomendation
## Jisun Lee_300301664

[1. Load dataset]

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

In [2]:
#pass in column names for each csv datafile and read them using pandas.
#Reading users file:
u_cols = ['user_id','age','sex','occupation','zip']
users = pd.read_csv('./dataset/u.user.csv', sep='|', names=u_cols)

#Reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings_data = pd.read_csv('./dataset/u.data.csv', sep='|', names=r_cols)

#Reading items file:
i_cols = ['movie_id', 'title' ,'release_date','video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('./dataset/u.item.csv', sep='|', names=i_cols)

In [3]:
#confirm the contents of each file
print(users.shape)
users.head()

(943, 5)


,user_id,age,sex,occupation,zip
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [4]:
print(ratings_data.shape)
ratings_data.head()

(100000, 4)


,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [5]:
print(items.shape)
items.head()

(1682, 24)


,movie_id,title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


[2. Analysis of the Dataset]

In [6]:
#Count user_id grouped by rating in train dat
ratings_data.groupby(["rating"])[["user_id"]].count()

,user_id
rating,
1,6110
2,11370
3,27145
4,34174
5,21201


In [7]:
#Count user_id grouped by movies in train data
ratings_data.groupby(["movie_id"])[["user_id"]].count()

,user_id
movie_id,
1,452
2,131
3,90
4,209
5,86
...,...
1678,1
1679,1
1680,1


[3. Build Popularity based recommendation]

In [8]:
movie_trend = pd.DataFrame(ratings_data.groupby("movie_id")["rating"].mean())
movie_trend['total number of ratings'] = pd.DataFrame(ratings_data.groupby('movie_id')['rating'].count()) 
movie_trend.sort_values("rating", ascending=False).sort_values("total number of ratings", ascending=False).head()


,rating,total number of ratings
movie_id,,
50,4.358491,583
258,3.803536,509
100,4.155512,508
181,4.007890,507
294,3.156701,485


In [9]:
#Calculate C and m
C = movie_trend["rating"].mean()
m = movie_trend["total number of ratings"].quantile(0.9)
print(C)
print(m)

3.076044508325179
169.0


In [10]:
#Filter out all qualified movies into a new DataFrame
top_movies = movie_trend.copy().loc[movie_trend["total number of ratings"] >= m]
movie_trend.shape, top_movies.shape

((1682, 2), (170, 2))

In [11]:
# Function that computes the weighted rating of each movie
def weighted_rating(x, m=m, C=C):
    v = x["total number of ratings"]
    R = x["rating"]
    return (v/(v+m) * R) + (m/(m+v) * C)

In [12]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
top_movies["score"] = top_movies.apply(weighted_rating, axis=1)
#Print the top 10 movies
top_movies = top_movies.sort_values("score", ascending=False)
top_movies[["rating", "total number of ratings", "score"]].head(10)

,rating,total number of ratings,score
movie_id,,,
50,4.358491,583,4.070281
318,4.466443,298,3.963279
64,4.445230,283,3.933300
127,4.283293,413,3.932735
98,4.289744,390,3.922811
174,4.252381,420,3.914858
483,4.456790,243,3.890416
100,4.155512,508,3.886044
12,4.385768,267,3.878100


[4. Build Collaborative filtering model]

In [13]:
#Calculate the number of unique users and movies
n_users = ratings_data.user_id.unique().shape[0]
n_users

943

In [14]:
n_movies = ratings_data.movie_id.unique().shape[0]
n_movies

1682

In [15]:
#Create a user-item matrix
ratings = np.zeros((n_users, n_movies))
ratings.shape

(943, 1682)

In [16]:
for row in ratings_data.itertuples():
    ratings[row[1]-1, row[2]-1] = row[3]
    
type(ratings)

numpy.ndarray

In [17]:
ratings

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [18]:
#Split train data and test data
from sklearn.model_selection import train_test_split

ratings_train, ratings_test = train_test_split(ratings, test_size=0.33, random_state=42)
ratings_train.shape, ratings_test.shape

((631, 1682), (312, 1682))

[4-1. User-based Collaborative Filtering]

In [19]:
#Measuring the similarity between all users
from sklearn.metrics.pairwise import cosine_similarity
user_distances = cosine_similarity(ratings_train)
user_distances

array([[1.        , 0.36475764, 0.44246231, ..., 0.02010641, 0.33107929,
        0.25638518],
       [0.36475764, 1.        , 0.42635255, ..., 0.06694419, 0.27339314,
        0.22337268],
       [0.44246231, 0.42635255, 1.        , ..., 0.06675756, 0.25424373,
        0.22320126],
       ...,
       [0.02010641, 0.06694419, 0.06675756, ..., 1.        , 0.04853428,
        0.05142508],
       [0.33107929, 0.27339314, 0.25424373, ..., 0.04853428, 1.        ,
        0.1198022 ],
       [0.25638518, 0.22337268, 0.22320126, ..., 0.05142508, 0.1198022 ,
        1.        ]])

In [20]:
user_pred = user_distances.dot(ratings_train)/np.array([np.abs(user_distances).sum(axis=1)]).T

In [21]:
from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

In [22]:
np.sqrt(get_mse(user_pred, ratings_train))

2.8075245308903365

In [23]:
np.sqrt(get_mse(user_pred, ratings_test))

2.9870546415652575

In [24]:
#Find the similarest with 5 neighbors
from sklearn.neighbors import NearestNeighbors
k = 5
neigh = NearestNeighbors(n_neighbors=k, metric="cosine")

In [25]:
neigh.fit(ratings_train)

NearestNeighbors(metric='cosine')

In [26]:
top_k_distances, top_k_users = neigh.kneighbors(ratings_train, return_distance=True)

In [27]:
top_k_users

array([[  0, 589, 155,  33, 364],
       [  1, 483, 339, 172, 188],
       [  2, 382, 560, 350, 155],
       ...,
       [628, 258, 242, 229, 494],
       [629, 378, 155, 589, 591],
       [630, 495, 201, 417, 603]], dtype=int64)

In [28]:
top_k_distances

array([[0.        , 0.38230161, 0.39990633, 0.40834169, 0.4100445 ],
       [0.        , 0.4625691 , 0.50677921, 0.50811827, 0.50882566],
       [0.        , 0.46538829, 0.48267976, 0.49176259, 0.49265099],
       ...,
       [0.        , 0.5764934 , 0.59340849, 0.64699606, 0.66472075],
       [0.        , 0.60496802, 0.6115226 , 0.62054374, 0.6229481 ],
       [0.        , 0.56320216, 0.60221688, 0.60314589, 0.6400121 ]])

In [29]:
#Evaluate the prediction using sum of weighted rating of 5 selected users
user_pred_k = np.zeros(ratings_train.shape)
for i in range(ratings_train.shape[0]):
    user_pred_k[i,:] = top_k_distances[i].T.dot(ratings_train[top_k_users][i])/ np.array([np.abs(top_k_distances[i].T).sum(axis=0)]).T

In [30]:
user_pred_k

array([[4.25618269, 2.49082621, 0.71654943, ..., 0.        , 0.        ,
        0.        ],
       [3.74418756, 0.        , 2.48873124, ..., 0.        , 0.        ,
        0.        ],
       [3.22293592, 2.98635211, 2.47648118, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [1.07143091, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [3.73945823, 2.48622549, 1.76969702, ..., 0.        , 0.        ,
        0.        ],
       [1.95357502, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [31]:
np.sqrt(get_mse(user_pred_k, ratings_train))

2.0922014531938316

In [32]:
np.sqrt(get_mse(user_pred_k, ratings_test))

3.054698791142718

[4-2. Item-based Collaborative Filtering]

In [33]:
#Set the number of movies as k and measuring the similarity btween items
k = ratings_train.shape[1]
neigh = NearestNeighbors(n_neighbors=k, metric="cosine")
neigh.fit(ratings_train.T)

NearestNeighbors(metric='cosine', n_neighbors=1682)

In [34]:
item_distances, similar_item = neigh.kneighbors(ratings_train.T, return_distance=True)

In [35]:
item_distances.shape

(1682, 1682)

In [36]:
similar_item

array([[   0,   49,  120, ..., 1576, 1569, 1679],
       [   1,  232,  160, ..., 1524, 1331, 1679],
       [   2,  762,  409, ..., 1298, 1423, 1680],
       ...,
       [   0, 1127, 1126, ...,  553,  577, 1681],
       [1680, 1671, 1422, ...,  735,  747, 1681],
       [1681, 1596,  766, ...,  700,  715, 1680]], dtype=int64)

In [37]:
#Evaluate the prediction
item_pred = ratings_train.dot(item_distances)/np.array([np.abs(item_distances).sum(axis=1)])

In [38]:
item_pred.shape

(631, 1682)

In [39]:
np.sqrt(get_mse(item_pred, ratings_train))

3.3784093354963565

In [40]:
np.sqrt(get_mse(item_pred, ratings_test))

3.5098535318342443